In [1]:
import os
import django

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "quanta.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [2]:
from apps.orders import models as order_models

In [3]:
import time
from django.db import transaction

In [4]:
import pandas as pd
from datetime import datetime, timedelta
from decimal import Decimal
from django.utils import timezone
from apps.orders.models import (
    Supplier,
    Category,
    Item,
    Batch,
    Organization,
    OrganizationUser,
)

In [5]:
for org in order_models.Organization.objects.all():
    print(org.name, org.id)

MAXIMAL SARL a14eb772-e4ae-40f1-a836-b7216bd30870
Docta Shop a8fbfab1-1cb2-461c-92e4-976dadcafdc3
DATAVITE SARL f017f9fc-cf47-4911-ba27-563151e15607
ELECBI SERVICES 546935a5-7e2f-4a01-ba3d-2d8ded8882ca
Tacynt Digital Agency 966a67f9-65e6-467d-8de8-fb954d2c16f7
Agri Distribution 69393ba2-3fbd-4ca8-a2d8-2e75ff14093d
CANTINE ISSI f438ab3d-5be5-40fd-b5ec-b16d00e32626


In [6]:
organization_id = "f438ab3d-5be5-40fd-b5ec-b16d00e32626"
organization_user_id = "c3d245b4-1159-4fbb-b1c3-7edb45ef917f"
supplier_id = "9d7e1561-c8ca-4843-b445-4d820ce3f5dc"

In [7]:
organization = order_models.Organization.objects.filter(id=organization_id)
organization_user = order_models.OrganizationUser.objects.filter(id=organization_user_id)
supplier = order_models.Supplier.objects.filter(id=supplier_id)

In [12]:
# order_models.Transaction.objects.filter(organization_id=organization_id).delete()
# order_models.Facturation.objects.filter(organization_id=organization_id).delete()
# order_models.Batch.objects.filter(organization_id=organization_id).delete()
# order_models.Item.objects.filter(organization_id=organization_id).delete()
# order_models.Category.objects.filter(organization_id=organization_id).delete()

In [13]:
import pandas as pd
import random
from decimal import Decimal
from datetime import datetime, timedelta
from django.db import transaction

def simple_feed_data(organization_id, organization_user_id, supplier_id):
    """
    Efficient bulk import:
    - Removes duplicate articles before any DB call
    - Uses bulk_create for Category, Item, Batch
    - Zero per-row DB queries
    """

        # Load Excel data
    data = pd.read_csv("demo/cantine_issi.csv", sep=";")

    # Drop rows missing required fields
    data = data.dropna(subset=["NATURE", "ARTICLES", "PRIX"])
    print(data)

    # Remove duplicates by article name (keep first occurrence)
    data = data.drop_duplicates(subset=["ARTICLES"], keep="first")

    # Prepare base tuples for processing
    all_data = list(zip(data["NATURE"], data["ARTICLES"], data["PRIX"]))
    
    # === 1️⃣ Create Categories ===
    category_names = sorted(set([cat for cat, _, _ in all_data]))
    categories_to_create = [
        Category(
            organization_id=organization_id,
            name=name,
            is_active=True,
            quanta={}
        )
        for name in category_names
    ]

    # Bulk create (ignore conflicts)
    server_categories = Category.objects.bulk_create(categories_to_create, ignore_conflicts=True)

    # Fetch category map {name: Category instance}
    categories = {c.name: c for c in server_categories}
    
    #print(categories)

    # === 2️⃣ Prepare Items ===
    items_to_create = []
    for category_name, item_name, price in all_data:
        items_to_create.append(Item(
            organization_id=organization_id,
            category=categories[category_name],
            name=item_name,
            alert_quantity=5,
            is_active=True,
            quanta=f"Q{random.randint(100000, 999999)}",
        ))

    server_items = Item.objects.bulk_create(items_to_create, ignore_conflicts=True)

    # Fetch all items for batch creation
    items = {i.name: i for i in server_items}

    # === 3️⃣ Prepare Batches ===
    batches_to_create = []
    for category, item_name, price in all_data:
        item = items.get(item_name)
        if item:
            batches_to_create.append(Batch(
                organization_id=organization_id,
                item=item,
                batch_number=f"B{random.randint(100000, 999999)}",
                supplier_id=supplier_id,
                received_date=datetime.now().date(),
                expiration_date=datetime.now().date() + timedelta(days=10000),
                purchase_price=Decimal("50.00"),
                facturation_price=Decimal(price),
                quantity=random.randint(10, 100),
                last_checked=datetime.now(),
                last_maintainer_id=organization_user_id,
                is_active=True,
                quanta={}
            ))

    # === 4️⃣ Bulk create all batches ===
    Batch.objects.bulk_create(batches_to_create, ignore_conflicts=False)

    print(f"✅ Loaded {len(items_to_create)} items across {len(categories)} categories.")
    return len(items_to_create)


In [14]:
with transaction.atomic():
       
    simple_feed_data(organization_id, organization_user_id, supplier_id)

                                    ARTICLES         NATURE  PRIX
0                                       PAIN  PETIT DEJEUNE   100
1                                       PAIN  PETIT DEJEUNE   150
2                              1 OEUF SIMPLE  PETIT DEJEUNE   150
3                           1 OEUF SPAGHETTI  PETIT DEJEUNE   250
4                          2 OEUFS SPAGHETTI  PETIT DEJEUNE   400
5                                    SARDINE  PETIT DEJEUNE   150
6                                MAYAUNNAISE  PETIT DEJEUNE   100
7   UNE TASSE DE LAIT SIMPLE/MATINAL/NESCAFE  PETIT DEJEUNE   200
8                         ERU + WATERFUFU NB  PETIT DEJEUNE   500
9                                  UNE BOULE          REPAS   300
10                           VIANDE DE BOEUF          REPAS   200
11                             PEAU DE BOEUF          REPAS   100
12             OKOK + MANIOC/BATON DE MANIOC          REPAS   700
13                                 RIZ SAUTE          REPAS   500
14        

/home/liedji/.local/share/virtualenvs/cameinet-IPrkOpQs/lib/python3.10/site-packages/django/db/models/fields/__init__.py:1595: RuntimeWarning: DateTimeField Batch.last_checked received a naive datetime (2025-12-02 07:18:01.552262) while time zone support is active.
  warnings.warn(
/home/liedji/.local/share/virtualenvs/cameinet-IPrkOpQs/lib/python3.10/site-packages/django/db/models/fields/__init__.py:1595: RuntimeWarning: DateTimeField Batch.last_checked received a naive datetime (2025-12-02 07:18:01.552377) while time zone support is active.
  warnings.warn(
/home/liedji/.local/share/virtualenvs/cameinet-IPrkOpQs/lib/python3.10/site-packages/django/db/models/fields/__init__.py:1595: RuntimeWarning: DateTimeField Batch.last_checked received a naive datetime (2025-12-02 07:18:01.552441) while time zone support is active.
  warnings.warn(
/home/liedji/.local/share/virtualenvs/cameinet-IPrkOpQs/lib/python3.10/site-packages/django/db/models/fields/__init__.py:1595: RuntimeWarning: DateTime

✅ Loaded 30 items across 2 categories.
